# Employee Salary Prediction

## Problem Statement
Develop a regression model using Artificial Neural Network (ANN) to predict employee salaries based on various features including credit score, geography, gender, age, tenure, balance, number of products, credit card status, active membership, and churn status. This model can help in salary benchmarking and compensation planning.


In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the data

data = pd.read_csv("dataset/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data

data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data.shape

(10000, 11)

In [5]:
# Encode categorical variables

label_encoder = LabelEncoder()
label_encoder_gender = label_encoder.fit_transform(data["Gender"])
data['Gender'] = label_encoder_gender

In [7]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
# OneHot Encode 'Geography'

onehot_encoder = OneHotEncoder()
onehot_encoder_geography = onehot_encoder.fit_transform(data[['Geography']]).toarray()
onehot_encoder_geography_df = pd.DataFrame(onehot_encoder_geography, columns=onehot_encoder.get_feature_names_out(['Geography']))
onehot_encoder_geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
combined_data = pd.concat([data.drop('Geography', axis=1), onehot_encoder_geography_df], axis=1)
combined_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [16]:
# Split the data into features and target.

x = combined_data.drop('EstimatedSalary', axis=1)
y = combined_data['EstimatedSalary']

# Split the data into training and testing sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=50)


In [17]:
# Scale the features

standard_scaler = StandardScaler()
x_train = standard_scaler.fit_transform(x_train)
x_test = standard_scaler.transform(x_test)

In [20]:
# Save the scaler for later use (encoders are saved from the churn prediction notebook)

with open('saved_models/regression_scaler.pkl', 'wb') as f:
    pickle.dump(standard_scaler, f)


In [23]:
model =  Sequential(
    [
        Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1,) #Output layer for regression.
    ]
)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# Set up early stopping.

early_stopping_fallback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [28]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    callbacks=[early_stopping_fallback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 13333426176.0000 - mae: 100078.2734 - val_loss: 13211545600.0000 - val_mae: 99737.4688
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 13191342080.0000 - mae: 99366.5391 - val_loss: 12923086848.0000 - val_mae: 98294.0312
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 12682872832.0000 - mae: 96832.3281 - val_loss: 12173919232.0000 - val_mae: 94538.1562
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 11664120832.0000 - mae: 91791.9297 - val_loss: 10902941696.0000 - val_mae: 88180.3359
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 10165768192.0000 - mae: 84330.2969 - val_loss: 9230484480.0000 - val_mae: 79752.2578
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 8390463488.0000 - mae: 75464.7812 - val_loss: 7425198592.0000 - val_mae: 70561.5469
Epoch 7/100
250/250 [===============

In [29]:
# Evaluate the model

loss, mae = model.evaluate(x_test, y_test)
print(f"Mean Absolute Error on test set: {mae:.4f}")


63/63 [==============================] - 0s 978us/step - loss: 3272751104.0000 - mae: 49242.7969
Mean Absolute Error on test set: 49242.7969


In [30]:
model.save('saved_models/regression_model.h5')

/home/murli/murli/git_projects/employee_churn_modeling/myenv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
